In [ ]:
%load_ext autoreload
%autoreload 1

# Import required libararies

In [ ]:
import mlflow

import pandas as pd

# Load the evaluation dataset

In [ ]:
import ast


def parse_workup(workup_str: str) -> dict:
    """Parse JSON strings into actual dictionaries."""
    return ast.literal_eval(workup_str)


def to_evaluation_format(row: pd.Series) -> dict:
    """Convert the DataFrame into the required evaluation format."""
    return {
        "inputs": {"scenario": row["Rephrased"]},
        "expectations": {
            "rating": row["Synthetic_life_rating"],
            "workup": parse_workup(row["Synthetic_life_rating_workup"]),
        },
        "tags": {"id": row["DeID"]},
    }

data = pd.read_csv("../data/rating_data.csv")

# Define systems to compare

In [ ]:
from typing import Any, Protocol


class PredictionSystem(Protocol):
    def predict(self, scenario: str) -> Any: ...

In [ ]:
from pydantic import BaseModel, Field
from workflow2reasoning.core.agents.underwriting import UnderwritingAgent


class ConditionRating(BaseModel):
    name: str
    condition_rating: str


class WorkflowUWResponse(BaseModel):
    rating: str | None = None
    conditions: list[ConditionRating] = Field(default_factory=list)
    error: str | None = None


class WorkflowSystem(PredictionSystem):
    def __init__(self, model_name: str):
        self.agent = UnderwritingAgent().compile()
        self.model_name = model_name

    @mlflow.trace
    def predict(self, scenario: str) -> WorkflowUWResponse:
        mlflow.update_current_trace(tags={"id": id})
        try:
            response = self.agent.invoke(
                {"medical_summary": scenario},
                config={
                    "max_concurrency": 4,
                    "recursion_limit": 300,
                    "configurable": {
                        "model_name": self.model_name,
                    },
                },
            )
            response = WorkflowUWResponse(
                rating=response["overall_rating"]["life_rating"],
                conditions=[
                    ConditionRating(
                        name=impairment,
                        condition_rating=trace["rating"]["co-morbidity"],
                    )
                    for impairment, trace in response["trace"].items()
                ],
            )
        except Exception as e:
            response = WorkflowUWResponse(error=str(e))
        return response

In [ ]:
from uw_baseline.agent import get_underwriter, UnderwritingResponse
from langchain.chat_models import init_chat_model
from langchain_core.rate_limiters import InMemoryRateLimiter


class ReActResponse(UnderwritingResponse):
    rating: str | None = None
    error: str | None = None


class ReActSystem(PredictionSystem):
    def __init__(self, model_name: str):
        rate_limiter = None
        if rate_settings := self.get_rate_limit_settings(model_name):
            rate_limiter = InMemoryRateLimiter(
                requests_per_second=rate_settings["requests_per_second"],
                check_every_n_seconds=0.1,
                max_bucket_size=rate_settings["max_bucket_size"],
            )
        self.model = init_chat_model(model_name, rate_limiter=rate_limiter)
        self.agent = get_underwriter(model=self.model)

    @mlflow.trace
    def predict(self, scenario: str) -> UnderwritingResponse | dict:
        try:
            response = self.agent.invoke(
                {"messages": [("user", scenario)]},
                {"config": {"recursion_limit": 50, "retry_limit": 10}},
            )
        except Exception as e:
            return ReActResponse(rating="", conditions=[], error=str(e))
        return response["structured_response"]

    def get_rate_limit_settings(self, model_name: str) -> dict:
        settings = {
            "azure_openai:gpt-4o": {"requests_per_second": 40, "max_bucket_size": 42},
            "azure_openai:gpt-4o-mini": {
                "requests_per_second": 18,
                "max_bucket_size": 19,
            },
            "azure_openai:gpt-4.1": {"requests_per_second": 2, "max_bucket_size": 2},
            "azure_openai:gpt-4.1-mini": {
                "requests_per_second": 46,
                "max_bucket_size": 48,
            },
            "azure_openai:gpt-4.1-nano": {
                "requests_per_second": 41,
                "max_bucket_size": 44,
            },
            "azure_openai:gpt-5": {"requests_per_second": 80, "max_bucket_size": 84},
            "azure_openai:gpt-5-mini": {"requests_per_second": 3, "max_bucket_size": 4},
            "azure_openai:o1-mini": {"requests_per_second": 3, "max_bucket_size": 4},
        }
        return settings.get(
            model_name.replace("azure_openai:", ""), None
        )  # Default to empty dict if model not found

# Define Experiment

In [ ]:
import mlflow
import pandas as pd
from evals.scorers import rating_is_exact_match, condition_details_match


def run_evaluations(
    data: pd.DataFrame,
    change_id: str,
    change_description: str,
    models: list,
    systems: dict[str, PredictionSystem],
):
    for model_name in models:
        for system_name, system_cls in systems.items():
            # System to evaluate
            system = system_cls(model_name=model_name)
            # Run Name
            provider, model = model_name.split(":")
            run_name = f"{system_name}-{model}-{change_id}"
            with mlflow.start_run(
                run_name=run_name,
                description=f"Evaluating {system_name} with {model} for the following changes: {change_description}",
                nested=True,
            ) as run:
                mlflow.log_table(data, "ground_truth.json")
                mlflow.log_params(
                    {
                        "change_id": change_id,
                        "change_description": change_description,
                        "system": system_name,
                        "model": model,
                        "provider": provider,
                        "num_samples": len(data),
                    }
                )
                results = mlflow.genai.evaluate(
                    data=data.apply(to_evaluation_format, axis=1).to_list(),
                    predict_fn=system.predict,
                    scorers=[rating_is_exact_match, condition_details_match],
                )
                print(f"Completed run: {run.info.run_id} - {run_name}")
                print(
                    f"Results: {'\n'.join([f'{k}: {v}' for k, v in results.metrics.items()])}"
                )
    print("All evaluations completed.")

# Run the experiments

In [ ]:
change_id = "docs"
change_description = ""  # What did you change
n = 3  # data.shape[0] for all data
systems = {
    "reAct": ReActSystem,
    "workflow": WorkflowSystem,
}
models = [
    # Format for this list is <provider>:<model_name>
    # See https://reference.langchain.com/python/langchain/models/#langchain.chat_models.init_chat_model for supported providers.
    # Ensure the right environment variables are set for the provider you choose. See README.md for example for OpenAI and Azure OpenAI
    "openai:gpt-4o",
    "openai:gpt-4o-mini",
    "openai:gpt-4.1",
    "openai:gpt-4.1-mini",
    "openai:gpt-4.1-nano",
    "openai:gpt-5",
    "openai:gpt-5-mini",
]
run_evaluations(data[:n], change_id, change_description, models, systems)